# Load data

In [35]:
%load_ext autoreload
%autoreload 2
from tqdm.notebook import tqdm
import logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
file = open("data/python.txt", "r")
files = []
corpus_file = ''
for line in file:
    if "EOF" not in line:
        corpus_file += line
    else:
        files.append(corpus_file)
        corpus_file = ''

In [37]:
print(files[0].replace("\\",""))

Python Syntax
Last update on February 28 2020 12:16:01 (UTC/GMT +8 hours)
Introduction
A Python program is read by a parser. Python was designed to be a highly readable language. The syntax of the Python programming language is the set of rules which defines how a Python program will be written.

Python Line Structure:

A Python program is divided into a number of logical lines and every logical line is terminated by the token NEWLINE. A logical line is created from one or more physical lines.
A line contains only spaces, tabs, formfeeds possibly a comment, is known as a blank line, and Python interpreter ignores it.
A physical line is a sequence of characters terminated by an end-of-line sequence (in windows it is called CR LF or return followed by a linefeed and in Unix, it is called LF or linefeed). See the following example.

Python Line structure
 

Comments in Python:

A comment begins with a hash character(#) which is not a part of the string literal and ends at the end of the p

# Preprocess data

In [38]:
from utils import get_regex_expression, preprocess_data

In [39]:
regex = get_regex_expression()
train_data =  [preprocess_data(file, regex) for file in tqdm(files)]

In [40]:
print(train_data[0])

['python', 'syntax', 'last', 'update', 'on', 'february', 'utc', 'gmt', 'hours', 'introduction', 'python', 'program', 'is', 'read', 'by', 'parser', 'python', 'was', 'designed', 'to', 'be', 'highly', 'readable', 'language', 'the', 'syntax', 'of', 'the', 'python', 'programming', 'language', 'is', 'the', 'set', 'of', 'rules', 'which', 'defines', 'how', 'python', 'program', 'will', 'be', 'written', 'python', 'line', 'structure', 'python', 'program', 'is', 'divided', 'into', 'number', 'of', 'logical', 'lines', 'and', 'every', 'logical', 'line', 'is', 'terminated', 'by', 'the', 'token', 'newline', 'logical', 'line', 'is', 'created', 'from', 'or', 'more', 'physical', 'lines', 'line', 'contains', 'only', 'spaces', 'tabs', 'formfeeds', 'possibly', 'comment', 'is', 'known', 'as', 'blank', 'line', 'and', 'python', 'interpreter', 'ignores', 'it', 'physical', 'line', 'is', 'sequence', 'of', 'characters', 'terminated', 'by', 'an', 'end', 'of', 'line', 'sequence', 'in', 'windows', 'it', 'is', 'called'

# Laguage Modeling

In [76]:
from gensim.models import Word2Vec, Phrases
import multiprocessing
number_cpus = multiprocessing.cpu_count()

In [77]:
bigrams = Phrases(train_data, threshold=5, min_count=1)

2020-06-27 23:06:06,218 : INFO : collecting all words and their counts
2020-06-27 23:06:06,220 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2020-06-27 23:06:06,270 : INFO : collected 9878 word types from a corpus of 16154 words (unigram + bigrams) and 19 sentences
2020-06-27 23:06:06,270 : INFO : using 9878 counts as vocab in Phrases<0 vocab, min_count=1, threshold=5, max_vocab_size=40000000>


In [79]:
train_data = list(bigrams[train_data])

In [80]:
parameters = {"min_count":0, "size":300, "sg":1, "window":15, "iter":110,
                   "sample": 1e-5, "hs": 0, "negative": 25, "ns_exponent": 0.5,
                   "workers": number_cpus}

In [81]:
sg_model = Word2Vec(**parameters)
sg_model.build_vocab(train_data, progress_per=100000)

2020-06-27 23:06:13,413 : INFO : collecting all words and their counts
2020-06-27 23:06:13,414 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-06-27 23:06:13,416 : INFO : collected 2598 word types from a corpus of 11839 raw words and 19 sentences
2020-06-27 23:06:13,417 : INFO : Loading a fresh vocabulary
2020-06-27 23:06:13,424 : INFO : effective_min_count=0 retains 2598 unique words (100% of original 2598, drops 0)
2020-06-27 23:06:13,425 : INFO : effective_min_count=0 leaves 11839 word corpus (100% of original 11839, drops 0)
2020-06-27 23:06:13,445 : INFO : deleting the raw counts dictionary of 2598 items
2020-06-27 23:06:13,446 : INFO : sample=1e-05 downsamples 2598 most-common words
2020-06-27 23:06:13,447 : INFO : downsampling leaves estimated 1843 word corpus (15.6% of prior 11839)
2020-06-27 23:06:13,456 : INFO : estimated required memory for 2598 words and 300 dimensions: 7534200 bytes
2020-06-27 23:06:13,457 : INFO : resetting layer weights


In [82]:
sg_model.train(train_data, total_examples=sg_model.corpus_count, 
                  epochs=sg_model.epochs)

2020-06-27 23:06:15,014 : INFO : training model with 8 workers on 2598 vocabulary and 300 features, using sg=1 hs=0 sample=1e-05 negative=25 window=15
2020-06-27 23:06:15,020 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-06-27 23:06:15,022 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-06-27 23:06:15,024 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-06-27 23:06:15,028 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-06-27 23:06:15,032 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-06-27 23:06:15,032 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-06-27 23:06:15,072 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-06-27 23:06:15,162 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-06-27 23:06:15,163 : INFO : EPOCH - 1 : training on 11839 raw words (1824 effective words) took 0.1s, 12513 effectiv

(202793, 1302290)

In [93]:
sg_model.wv.most_similar("dict")

[('keys', 0.9074428677558899),
 ('dictionary', 0.9041739106178284),
 ('zip', 0.897850751876831),
 ('test_whether', 0.8793338537216187),
 ('blue_red', 0.8770774602890015),
 ('counter', 0.8758074641227722),
 ('dict_items', 0.8747591972351074),
 ('counter_colors', 0.8724720478057861),
 ('yellow_counter', 0.8708021640777588),
 ('filters', 0.8697898387908936)]